# Chapter 6. 디스크립터

## 개요
### 디스크립터 메커니즘
+ 최소 두 개의 클래스가 필요
+ 디스크립터 프로토콜 중 하나를 구현한 클래스 인트턴스를 클래스 속성으로 포함해야 함
    + \_\_get\_\_
    + \_\_set\_\_
    + \_\_delete\_\_
    + \_\_set_name\_\_

In [1]:
""" 디스크립터는 __get__ 매직 메서드의 결과를 반환함 """

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DescriptorClass:
    def __get__(self, instance, owner): # owner의 인스턴스와 클래스를 받음
        if instance is None:
            return self
        logger.info(
            "Call: %s.__get__(%r, %r)", 
            self.__class__.__name__,
            instance,
            owner,
        )
        return instance


class ClientClass:
    descriptor = DescriptorClass()

In [2]:
client = ClientClass()
client.descriptor

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x7fdd740bb2d0>, <class '__main__.ClientClass'>)


In [3]:
client.descriptor is client

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x7fdd740bb2d0>, <class '__main__.ClientClass'>)


True

### 디스크립터 프로토콜의 매서드 
#### \_\_get\_\_(self, instance, owner)
+ owner가 있는 경우는 클래스에서 직접 호출하는 경우 instance가 None이기 때문

In [4]:
""" 디스크립터를 인스턴스로 호출할 때와 클래스 속성으로 호출할 때의 차이점 """

class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self, owner
        logger.info("Call: %s.__get__(%r, %r)",
                    self.__class__.__name__, instance, owner)
        return instance
    
class ClientClass:
    descriptor = DescriptorClass()

In [5]:
ClientClass().descriptor

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x7fdd7404f1d0>, <class '__main__.ClientClass'>)


In [6]:
ClientClass.descriptor # instance가 None이므로 self를 반환

(<__main__.DescriptorClass at 0x7fdd7404f490>, __main__.ClientClass)

#### \_\_set\_\_(self, instance, value)
+ 디스크립터에 값을 할당할 때 호출되며, \_\_set\_\_( ) 메서드를 구현한 디스크립터에 대해서만 활성화
+ \_\_set\_\_이 구현되지 않은 경우 client.descriptor = "value" 에서 descriptor자체를 덮어쓸 수 있으므로 주의 필요

In [7]:
from typing import Callable, Any # 형 힌트 지원
                                 # Callable[[Arg1Type, Arg2Type], ReturnType] 

class Validation: # 검증함수와 에러 메세지만을 가지고 있고 value는 __set__ 호출 시 전달받음
    """A configurable validation callable."""

    def __init__(
        self, validation_function: Callable[[Any], bool], error_msg: str 
    ) -> None:
        self.validation_function = validation_function
        self.error_msg = error_msg

    def __call__(self, value): # client.descriptor = 42 에서 42
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")


class Field:
    """A class attribute with validation functions configured over it."""

    def __init__(self, *validations):
        self._name = None
        self.validations = validations

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def validate(self, value):
        for validation in self.validations:
            validation(value)

    def __set__(self, instance, value): # client.descriptor = 42 처럼 대입 시 실행됨
        self.validate(value) # 여기서 raise ValueError 안된다면 다음 줄에서 대입됨
        instance.__dict__[self._name] = value


class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), "is not a number"),
        Validation(lambda x: x >= 0, "is not >= 0"),
    )

In [8]:
client = ClientClass()
client.descriptor = 42
print(client.descriptor)

client.descriptor = -42

42


ValueError: -42 is not >= 0

In [9]:
client.descriptor = "string"

ValueError: 'string' is not a number

#### \_\_delete\_\_(self, instance)
+ name과 email을 None으로 직접 설정하지 못하도록 \_\_set\_\_  함수 구현
+ \_\_ delete\_\_ 를 통해서만 email의 None설정 가능

In [10]:
class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, user, value):
        if value is None:
            raise ValueError(f"{self._name} can't be set to None")
        user.__dict__[self._name] = value

    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(
                f"User {user!s} doesn't have {self.permission_required} "
                "permission"
            )


class User:
    """Only users with "admin" privileges can remove their email address."""

    email = ProtectedAttribute(requires_role="admin")

    def __init__(
        self, username: str, email: str, permission_list: list = None
    ) -> None:
        self.username = username
        self.email = email
        self.permissions = permission_list or []

    def __str__(self):
        return self.username

In [11]:
admin = User("root", "root@d.com", ["admin"])
user = User("user", "user@d.com", ["email", "helpdesk"])

In [12]:
del admin.email
admin.email is None

True

In [13]:
del user.email

ValueError: User user doesn't have admin permission

#### \_\_set_name\_\_(self, ower, name)
+ 디스크립터가 처리하려는 속성의 이름을 알려줌
+ 클래스 데코레이터, 메타클래스를 사용해야하는 종전의 방법을 개선함(3.6 부터)

## 디스크립터의 유형
+ \_\_set\_\__, \_\_delete\_\__ 메서드를 구현했다면 데이터 디스크립터로 구분됨
+ \_\_get\_\__만 구현했다면 비데이터 디스크립터로 구분됨

### 비데이터 디스크립터
+ set을 구현하지 않았으므로 객체의 사전이 우선 순위를 갖고 그 다음에 디스크립터가 실행됨

In [14]:
class NonDataDesciptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42
    

class ClientClass:
    descriptor = NonDataDesciptor()   

In [15]:
client = ClientClass()
client.descriptor

42

In [16]:
vars(client) # 내부 딕셔너리에 아무것도 등록되지 않음

{}

In [17]:
client.descriptor = 99
vars(client)

{'descriptor': 99}

In [18]:
client.descriptor 

99

In [19]:
del client.descriptor
client.descriptor # 비데이터 디스크립터의 경우 내부 딕셔너리보다 후순위로 검색되는 것을 알 수 있음

42

### 데이터 디스크립터
+ set을 구현했으므로 객체 사전보다 높은 우선순위를 갖게 됨
+ 디스크립터의 set에서 setattr()이나 할당 표현식을 직접 사용하면 무한루프가 발생하므로 주의가 필요함

In [20]:
class DataDesciptor:
    
    def __init__(self):
        self._name = None
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42
    
    def __set__(self, instance, value):
        instance.__dict__[self._name] = value

    def __set_name__(self, ower, name):
        self._name = name    

        
class ClientClass:
    descriptor = DataDesciptor()

In [21]:
client = ClientClass()
client.descriptor

42

In [22]:
vars(client) # 내부 딕셔너리에 아무것도 등록되지 않음

{}

In [23]:
client.descriptor = 99
vars(client)

{'descriptor': 99}

In [24]:
client.descriptor # 내부 딕셔너리 보다도 우선순위가 높음, delete가 구현되어 있지 않으므로 del을 사용할 수 없음

42

In [25]:
client.__dict__["descriptor"]

99

## 디스크립터 실전
+ 디스크립터는 클라이언트 클래스와 완전히 독립적이여야 함
+ 어떠한 비즈니스 로직도 포함되어 있지 않아야 함
+ 라이브러리, 프레임워크 또는 내부 API를 정의해야 함

In [26]:
""" 여행지를 누적하는 디스크립터 """

class HistoryTracedAttribute:
    """Trace the values of this attribute into another one given by the name at
    ``trace_attribute_name``.
    """

    def __init__(self, trace_attribute_name: str) -> None:
        self.trace_attribute_name = trace_attribute_name  # cities_visited
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name] # current_city

    def __set__(self, instance, value):
        self._track_change_in_value_for_instance(instance, value)
        instance.__dict__[self._name] = value # current_city

    def _track_change_in_value_for_instance(self, instance, value):
        self._set_default(instance)
        if self._needs_to_track_change(instance, value):
            instance.__dict__[self.trace_attribute_name].append(value) # 키값의 리스트에 여행을 추가

    def _needs_to_track_change(self, instance, value) -> bool:
        """Determine if the value change needs to be traced or not.
        Rules for adding a value to the trace:
            * If the value is not previously set (it's the first one).
            * If the new value is != than the current one.
        """
        try:
            current_value = instance.__dict__[self._name]
        except KeyError: # 키가 없으면 발생
            return True
        return value != current_value # 현재 키값도 새로운 키값이 다른 경우에만 리스트에 추가하도록 True를 리턴함

    def _set_default(self, instance):
        instance.__dict__.setdefault(self.trace_attribute_name, []) # 키가 없으면 기본값으로 리스트를 반환


class Traveller:
    """A person visiting several cities.
    We wish to track the path of the traveller, as he or she is visiting each new city.
    """

    current_city = HistoryTracedAttribute("cities_visited")

    def __init__(self, name, current_city):
        self.name = name
        self.current_city = current_city

In [27]:
alice = Traveller("Alice", "Barcelona")
alice.current_city = "Paris"
alice.current_city = "Brussels"
alice.current_city = "Brussels"
alice.current_city = "Amsterdam"
alice.cities_visited

['Barcelona', 'Paris', 'Brussels', 'Amsterdam']

In [28]:
alice.current_city

'Amsterdam'

#### 클래스 데코레이터 피하기

In [29]:
from datetime import datetime
from functools import partial
from typing import Any, Callable


class BaseFieldTransformation:
    """Base class to define descriptors that convert values."""

    def __init__(self, transformation: Callable[[Any, str], str]) -> None:
        self._name = None
        self.transformation = transformation

    def __get__(self, instance, owner):
        if instance is None:
            return self
        raw_value = instance.__dict__[self._name]
        return self.transformation(raw_value)

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, instance, value):
        instance.__dict__[self._name] = value


ShowOriginal = partial(BaseFieldTransformation, transformation=lambda x: x) # transformation이 자동 설정됨
HideField = partial(BaseFieldTransformation, transformation=lambda x: "**redacted**")
FormatTime = partial(
    BaseFieldTransformation,
    transformation=lambda ft: ft.strftime("%Y-%m-%d %H:%M"),
)


class LoginEvent:
    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()

    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp

    def serialize(self):
        return {
            "username": self.username,
            "password": self.password,
            "ip": self.ip,
            "timestamp": self.timestamp,
        }


class BaseEvent:
    """Abstract the serialization and the __init__"""

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def serialize(self):
        return {
            attr: getattr(self, attr) for attr in self._fields_to_serialize()
        }

    def _fields_to_serialize(self):
        for attr_name, value in vars(self.__class__).items():
            if isinstance(value, BaseFieldTransformation): # '__module__, __doc__ 등은 제외하고 변환함
                yield attr_name


class NewLoginEvent(BaseEvent):
    """A class that takes advantage of the base to only define the fields."""

    username = ShowOriginal()
    password = HideField()
    ip = ShowOriginal()
    timestamp = FormatTime()

In [30]:
le = NewLoginEvent(username = "usr",
                   password = "secret password",
                   ip = "127.0.0.1",
                   timestamp = datetime(2016, 7, 20, 15, 45))

In [31]:
le.serialize()

{'username': 'usr',
 'password': '**redacted**',
 'ip': '127.0.0.1',
 'timestamp': '2016-07-20 15:45'}

### 파이썬 내부의 디스크립터 활용

In [32]:
""" 함수와 메서드 """

class Method:
    def __init__(self, name):
        self.name = name
        
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출, 인자는 {arg1}과 {arg2}")
        
class MyClass:
    method = Method("Internal call")

In [33]:
instance = MyClass()
Method("External call")(instance, "1st", "2nd")

External call: <__main__.MyClass object at 0x7fdd7402d450> 호출, 인자는 1st과 2nd


In [34]:
instance.method("1st", "2nd")

TypeError: __call__() missing 1 required positional argument: 'arg2'

In [35]:
""" 함수와 메서드, get을 구현하여 클래스 메소드로도 사용할 수 있도록 개선 """
from types import MethodType

class Method:
    def __init__(self, name):
        self.name = name
        
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출, 인자는 {arg1}과 {arg2}")
        
    def __get__(self, myclass_inst, myclass_cls):
        if myclass_inst == None:
            return self
        else:
            return MethodType(self, myclass_inst)
        
class MyClass:
    method = Method("Internal call")

In [36]:
instance = MyClass()
Method("External call")(instance, "1st", "2nd")
instance.method("1st", "2nd")

External call: <__main__.MyClass object at 0x7fdd6d7d0190> 호출, 인자는 1st과 2nd
Internal call: <__main__.MyClass object at 0x7fdd6d7d0190> 호출, 인자는 1st과 2nd
